<a href="https://colab.research.google.com/github/BirHic/Depth-Asena/blob/main/DepthV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ## **Asena PLUG / Interpolation**
 
`Kullanımı:`
- Google Drive üzerinde \"`DAIN`\" adlı bir klasör oluşturun.
- Google Drive üzerinde \"`videos`\" adlı bir klasör oluşturun.
 
- Önemli Not: Videoları \"`videos`\" klasörüne yükleyin. \"`DAIN`\" adlı klasöre hiçbir dosya yüklemeyin.

- `Önemli Not`: Sayfa yenilenirse yada bağlantı koparsa işlemleri en baştan yapmanız gerekecektir. Bunun önüne geçmek için çift ekran kullanabilirsiniz.

In [139]:
#@markdown # Gerekli Kurulum
 
#@markdown ## Input file
#@markdown Lütfen sadece `mp4` ve `gif` formatında video yükleyin. Sadece `720p` ve altı çözünürlüğe sahip videolar desteklenir. Sadece `input.mp4` yazan yeri yüklediğiniz video ismi ile değiştirin.
INPUT_FILEPATH = "videos/yourfile.mp4" #@param{type:"string"}
 
#@markdown ## Output file
#@markdown Sadece `output.mp4` yazan yeri belirlediğiniz bir isim ile değiştirin. 
OUTPUT_FILE_PATH = "DAIN/output.mp4" #@param{type:"string"}
 
 
#@markdown ## Hedef FPS
#@markdown Yüksek FPS değeri daha uzun süre işlem gerektirir
TARGET_FPS =  120#@param{type:"number"}
 
#@markdown `AŞAĞIDAKİ KODLARI BİLGİNİZ OLMADAN DOKUNMAYIN`
 
#@markdown ## Frame input directory
#@markdown Ana GDrive Root Dizininde, Elde Edilen PNG Dosyalarının Hangi Klasörün İçine oluşturulacağını seçin. Örnek PNG Adları 00001.png, 00002.png
FRAME_INPUT_DIR = '/content/DAIN/input_frames' #@param{type:"string"}
 
#@markdown ## Frame output directory
#@markdown Ana GDrive Root Dizininde, Oluşturulan PNG Dosyalarının Hangi Klasörün İçine Yerleştirileceğini seçin.
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' #@param{type:"string"}
 
#@markdown ## Seamless playback
#@markdown Perfect Loop İçin İlk ve Son Kareleri Birleştirir. Etkinleştirme için Tike Basın.
SEAMLESS = True #@param{type:"boolean"}
 
#@markdown ## Resize hotfix
#@markdown Interpolate frameleri, orijinal girdi çerçevelerine kıyasla biraz \ "kaymış / daha küçük \". Bu, Interpolate frame'lerini + 2px çözünürlüğe yeniden boyutlandırarak ve sonucu başlangıç ​​noktası (1,1) ile orijinal boyutuna kırparak kısmen hafifletilebilir. Bu düzeltme olmadan Interpolate, \ "titreşimli \" çıktı oluşturma eğilimindedir ve metin gibi statik öğelerle oldukça fark edilir.
#@markdown
#@markdown Bu düzeltme, daha yumuşak bir videoyu ayarlamak için bu tür efektleri daha az görünür kılmaya çalışır. CVPR 2018'in bu sorun için düzeltme olarak neyi kullandığını bilmiyorum, ancak orijinal, varsayılan test görüntülerinde böyle bir davranış gösteriyor. Daha ileri düzey kullanıcılar Interpolation yöntemini değiştirebilir. Cv2.INTER_CUBIC ve cv2.INTER_LANCZOS4 yöntemleri önerilir. Geçerli varsayılan değer cv2.INTER_LANCZOS4'tür.
RESIZE_HOTFIX = True #@param{type:"boolean"}
 
#@markdown ## Auto-remove PNG directory
#@markdown Ffmpeg video oluşturulduktan sonra çıktı PNG dizinini otomatik siler. PNG dosyalarını saklamak istiyorsanız bunu "False" olarak ayarlayın.
AUTO_REMOVE = True #@param{type:"boolean"}

# Modüller Yükleniyor...
Hiçbir modül bağlı olduğunuz internet üzerinden indirilmez. Data harcaması - Bellek kullanımı yapmaz.

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
# PyTorch ve SciPy Mödüllerinin en bilinen versiyonları yükleniyor...
 
!apt-get update
!pip install torch==1.13.1 torchvision==0.14.1+rocm5.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.10.0
 
print("Done!")

In [ ]:
# Interpolate Dosyaları Klonlanıyor...
 
%cd /content
!rm -rf DAIN
!git clone -b colab-compatibility --depth 1 https://github.com/AlphaGit/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

In [ ]:
# Bu Aşama 10-15 Dakika Sürecektir.
# Interpolate Yükleniyor.
 
%cd /content/DAIN/my_package/
!./build.sh
print("Done!")

In [ ]:
# Bu Aşama 5 Dakika Sürecektir.
# PyTorch Dosyaları Yükleniyor.
 
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Done!")

 - `Önemli Not:` Sayfayı yenilemeden, bağlantıyı kesmeden birden fazla işlem yapmak için yukarıdaki video ismini, hedef fps'i vs. değiştirdikten sonra aşağıdaki koddan itibaren çalıştırın.

In [118]:
# BU KODU BİRDEN FAZLA İŞLEMLERDE KULLANIN.
# İkinci bir videoyu Interpolate etmeden bu kodu kullanarak eski PNG dosyalarını silin.
if(AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*

In [ ]:
# Pre-trained Modeller İndiriliyor.
 
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [ ]:
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Done!")

In [140]:
 
# Videodaki FPS Değeri Hesaplanıyor.
 
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/
 
import os
filename = os.path.basename(INPUT_FILEPATH)
 
import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')
 
fps = cap.get(cv2.CAP_PROP_FPS)
 
if(fps/TARGET_FPS>0.5):
  print("DAHA YÜKSEK FPS DEĞERİ GİRİN. YENİ GİRDİĞİNİZ FPS DEĞERİ ORİJİNAL VİDEONUN 2 KATI OLMALIDIR")

In [ ]:
# ffmpeg çıkarılıyor. Yeni kareler için alan açılıyor.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'
 
%shell ffmpeg -i '/content/DAIN/{filename}' '{FRAME_INPUT_DIR}/%05d.png'
 
png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
from IPython.display import clear_output
clear_output()
 
pngs_generated_count = int(png_generated_count_command_result.output.strip())
 
import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)
 
print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")
 
# Alpha oranına bakılıyor
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")
 
# Belirlenen Alpha'lar siliniyor
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [ ]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN
 
!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

In [ ]:
# Dosyalar Bulunuyor, Orijinali ile Eşleşmesi İçin Sıkıştırılıyor...
 
import numpy as np
%cd {FRAME_OUTPUT_DIR}
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      cv2.imwrite(part_filename[0]+".png", crop)
 
%cd /content/DAIN

In [ ]:
# Video Oluşturuluyor...
 
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

 ## 30 Saniye bekledikten sonra videonuz `DAIN` klasöründe olacaktır. Drive'ı açarak kontrol edebilirsiniz.
#`Önemli Not`:
Eğer 'Bu video oynatılamıyor' hatası alıyorsanız lütfen biraz bekleyin. Hala Drive üzerinde açamıyorsanız 3. Parti uygulamalar ile açabilirsiniz. Videolari çok daha kolay bir şekilde oynamak için ise android üzerinde `Audio Video Factory`, PC üzerinde ise `WinX HD Video Conventer` uygulamalarını kullanabilirsiniz. IOS kullanıcılarının ek bir uygulamaya ihtiyacı yoktur.

 ` AŞAĞIDAKİ KOMUTLAR HER ZAMAN ÇALIŞMAYABİLİR. KULLANIMINI BİLENLER HARİCİ KODLARLA OYNAMAYIN`

In [ ]:
# [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

if (AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac